# IE2 Big Data Project (weilemar & vongrdir)

Im folgenden Jupyter Notebook wird der Code vorgestellt, welcher verwendet wurde um das [Homocide Report](https://www.kaggle.com/murderaccountability/homicide-reports) Dataset zu analysieren. Ziel dabei ist es ...

Als erster Schritt muss das Dataset über die folgende URL heruntergeladen werden:

In [ ]:
%%bash
wget -O homocide-reports.zip https://github.com/vongruenigen/IE2-Project/raw/master/data/homicide-reports.zip
unzip homocide-reports.zip

if [ ! -f database.csv ]
then
    echo 'ERROR: Could not fetch or extract homocide database!'
    exit 2
fi

Nachdem das Dataset heruntergeladen und entpackt ist muss daraus mittels des preprocessing Skripts (für Original siehe _scripts/preprocess_data.py_) das eigentliche Dataset mit One-Hot Encoded Vektoren verwendet werden. Da dieses Skript hier im Jupyter-Notebook nicht direkt aufgerufen werden kann müssen die Parameter in der Variable _argv_ gesetzt werden.

In [ ]:
import os
import sys
import h5py
import time
import numpy as np

from collections import defaultdict
from operator import itemgetter
from os import path

argv = ['database.csv', 'samples.h5']

if len(argv) < 2:
    print('ERROR: process_data.py <data-csv> <out-h5>')
    sys.exit(2)

data_path = argv[0]
out_path = argv[1]

# Columns we are going to ignore in the dataset
# because they're redundant or non-informative
STRIP_COLS = ('Record ID', 'Agency Name', 'Agency Code',
              'Year', 'Month', 'Record Source')

if path.isfile(out_path):
    os.remove(out_path)

with open(data_path, 'r') as data_f:
    with h5py.File(out_path) as out_f:
        # Read headings
        columns = data_f.readline().strip('\n').split(',')
        col_values = defaultdict(list)
        last_idx = 0

        print('The following columns are filtered: %s' % ', '.join(STRIP_COLS))

        # Find all unique values for each row in the dataset
        # and store them in col_values.
        for i, line in enumerate(data_f):
            sample_values = line.strip('\n').split(',')

            for c, v in zip(columns, sample_values):
                if c in STRIP_COLS: continue
                if v not in col_values[c]: col_values[c].append(v)

            last_idx = i

        print('The number of distinct values for each column are:\n')

        sum_lines = last_idx + 1
        sum_vec_entries = 0

        for c, v in col_values.items():
            print('  %s = %d' % (c, len(v)))
            sum_vec_entries += len(v)

        print('\nThe generated vectors will have a total of %d entries each' % sum_vec_entries)
        print('The dataset has %i samples\n' % sum_lines)

        data_f.seek(0)
        data_f.readline() # skip headings after seek(0)

        X = out_f.create_dataset('x', dtype='i8',
                                 shape=(sum_lines, sum_vec_entries))

        start_time = time.time()
        curr_idx = 0
        temp_x = []

        for i, line in enumerate(data_f):
            sample_values = line.strip('\n').split(',')
            sample_vec = np.zeros(sum_vec_entries)
            idx_offset = 0

            for c, v in zip(columns, sample_values):
                if c in STRIP_COLS: continue
                sample_vec[col_values[c].index(v)+idx_offset] = 1
                idx_offset += len(col_values[c])

            temp_x.append(sample_vec)

            if (i+1) % 100000 == 0 or (i+1) == sum_lines:
                temp_x = np.array(temp_x)
                np.random.shuffle(temp_x)

                print('Processed %i samples (%.1f%%)...' % (i+1, 100*(float(i+1)/sum_lines)))
                print('Storing collected data in h5py file...')

                X[curr_idx:curr_idx+temp_x.shape[0]] = temp_x
                curr_idx += temp_x.shape[0]

                print('\nStored data successfully! (Took %.2fs)' % (time.time() - start_time))
                start_time = time.time()
                temp_x = []

        print('Successfully stored preprocessed samples in: %s' % out_path)

Das Skript konvertiert alle Morde in _database.csv_ in One-Hot Encoded Vektoren und speichert diese in einer HDF5 Datei. Diese können dann verwendet werden um damit den _AutEncoder_ zu trainieren und danach Embeddings für jeden einzelnen Mord zu generieren.

Beim Preprocessen werden alle Spalten berücksichtigt ausser diejenigen, welche in der Liste STRIP_COLS explizit ausgeschlossen werden. Das ganz funktioniert so, dass zuerst für jede Spalte eruiert wird, wieviele unterschiedliche Werte es pro Spalte hat. Wenn für eine Spalte _n_ verschiedene Werte vorhanden sind, so werden für die Darstellung im One-Hot Vektor entsprechend _n_ Werte für diese Spalte benötigt. Die resultierenden Vektoren sind also 

$$\sum_{c \in Columns} \operatorname{classes}(c)$$

lang, wobei _C_ für die Menge aller Spalten und _classes_ für die Anzahl unterschiedlicher Werte für die Spalte _c_ steht. Pro Zeile und Spalte werden dann diejenigen Werte, welche in der jeweiligen Zeile stehen auf _1_ gesetzt, alle anderen werden auf _0_ belassen. Als nächste folgt der Code in [TensorFlow](https://www.tensorflow.org/), welcher für die Implementation des _AutoEncoder_ und des _VariationalAutoEncoder_ zuständig ist.

Prinzipell sind _AutoEncoder_ eine spezielle Art von Neuronalen Netzen (NN), welche dafür zuständig sind, eine effiziente Codierung der Eingabedaten zu lernen. Der Aufbau ist so, dass die Eingabedaten als One-Hote Encoded Vektoren (im Bild unten _x_) über den Input Layer in das NN eingespeist wird. Diese werden dann mithilfe einer Multiplikation mit einer Gewichts-Matrix in den Hidden Layer projeziert. Dieser ist im Falle von AutoEncodern **immer** kleiner wie der Input Layer, weil das NN ja eine effiziente Codierung der Eingabedaten lernen soll. Am Ende wird das NN mithilfe von Gradient-Descent mit dem _Adam_ Optimierer darauf trainiert, aus der codierten Darstellung der Eingabedaten (im Bild unten _z_) wieder die Eingabedaten _x_ zu rekonstruieren. Die generierten Darstellungen _z_ können dann als Embeddings der Eingabedaten in einem _m_-dimensionalen Vektorraum aufgefasst werden, wobei _m_ der grösse des Hidden Layer in der Mitte entspricht. Auf diese Embeddings können wir dann später Clustering-Algorithmen anwenden um festzustellen, welche Verbrechen im eingebetteten Vektor-Raum nahe beieinander liegen.

![AutoEncoder Struktur](https://upload.wikimedia.org/wikipedia/commons/2/28/Autoencoder_structure.png)

Der Unterschied eines "normalen" _AutoEncoder_ zu einem _VariationalAutoEncoder_ liegt darin, dass ...
Für eine gute Einführung in _VariationalAutoEncoder_ kann [dieses](http://kvfrans.com/variational-autoencoders-explained/) Tutorial hinzugezogen werden.

Wir werden alle Experimente mit beiden Varianten _AutoEncoder_ und _VariationalAutoEncoder_ durchführen. Unten folgt die Defintion der Modelle in Python mithilfe von TensorFlow.

In [ ]:
import tensorflow as tf

class AutoEncoder(object):
    def __init__(self, input_size, hidden_size, session):
        '''Initializes a new instance of the VariationalAutoencoder class.'''
        self.session = session
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weights = {}
        self.transfer_fn = tf.nn.softplus

        self.__initialize()
        self.__build()

    def __initialize(self):
        '''Initializes the weights needed to build the computational graph.'''
        weights = {}

        weights['weights_1'] = tf.get_variable('weights_1',[self.input_size, self.hidden_size])
        weights['bias_1'] = tf.Variable(tf.zeros([self.hidden_size], dtype=tf.float32))
        weights['weights_2'] = tf.Variable(tf.zeros([self.hidden_size, self.input_size], dtype=tf.float32))
        weights['bias_2'] = tf.Variable(tf.zeros([self.input_size], dtype=tf.float32))

        self.weights = weights

    def get_optimizer(self):
        '''Returns the optimizer for this instance.'''
        return self.optimizer

    def get_loss(self):
        '''Returns the loss function for this instance.'''
        return self.loss_fn

    def get_weights_and_biases(self):
        '''Returns the weights and biases of this instance.'''
        return self.weights

    def get_internal_representation(self):
        '''Returns the internal, embedded representation variables.'''
        return self.hidden

    def batch_fit(self, input):
        '''Fits the model to the given batch.'''
        loss, _ = self.session.run((self.loss_fn, self.optimizer),
                                   feed_dict={self.input: input})
        return loss

    def transform(self, input):
        return self.session.run(self.hidden, feed_dict={self.input: input})

    def __build(self):
        '''Builds the computational graph.'''
        self.input = tf.placeholder(tf.float32, [None, self.input_size])

        hidden_1_result = tf.matmul(self.input, self.weights['weights_1'])
        self.hidden = self.transfer_fn(tf.add(hidden_1_result,
                                              self.weights['bias_1']))

        reconstruction_result = tf.matmul(self.hidden, self.weights['weights_2'])
        self.reconstruction = tf.add(reconstruction_result, self.weights['bias_2'])

        diff = tf.subtract(self.reconstruction, self.input)
        self.loss_fn = 0.5 * tf.reduce_sum(tf.pow(diff, 2.0))
        self.optimizer_fn = tf.train.AdamOptimizer(learning_rate=0.001)
        self.optimizer = self.optimizer_fn.minimize(self.loss_fn)

class VariationalAutoencoder(object):
    def __init__(self, input_size, hidden_size, session):
        '''Initializes a new instance of the VariationalAutoencoder class.'''
        self.session = session
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weights = {}

        self.__initialize()
        self.__build()

    def __initialize(self):
        '''Initializes the weights needed to build the computational graph.'''
        weights = {}

        weights['weights_1'] = tf.get_variable('weights_1',[self.input_size, self.hidden_size])
        weights['log_sigma_weights_1'] = tf.get_variable('log_sigma_weights_1', [self.input_size, self.hidden_size])
        weights['bias_1'] = tf.Variable(tf.zeros([self.hidden_size], dtype=tf.float32))
        weights['log_sigma_bias_1'] = tf.Variable(tf.zeros([self.hidden_size], dtype=tf.float32))
        weights['weights_2'] = tf.Variable(tf.zeros([self.hidden_size, self.input_size], dtype=tf.float32))
        weights['bias_2'] = tf.Variable(tf.zeros([self.input_size], dtype=tf.float32))

        self.weights = weights

    def get_optimizer(self):
        '''Returns the optimizer for this instance.'''
        return self.optimizer

    def get_loss(self):
        '''Returns the loss function for this instance.'''
        return self.loss_fn

    def get_weights(self):
        '''Returns the weights of this instance.'''
        return self.weights

    def get_internal_representation(self):
        '''Returns the internal, embedded representation variables.'''
        return self.z

    def batch_fit(self, input):
        '''Fits the model to the given batch.'''
        loss, _ = self.session.run((self.loss_fn, self.optimizer),
                                   feed_dict={self.input: input})
        return loss

    def transform(self, input):
        return self.session.run(self.z_mean, feed_dict={self.input: input})

    def partial_fit(self, X):
        loss, opt = self.sess.run((self.loss_fn, self.optimizer), feed_dict={self.x: X})
        return loss

    def transform(self, X):
        return self.sess.run(self.z_mean, feed_dict={self.x: X})

    def __build(self):
        '''Builds the computational graph.'''
        self.input = tf.placeholder(tf.float32, [None, self.input_size])

        hidden_1_result = tf.matmul(self.input, self.weights['weights_1'])
        self.z_mean = tf.add(hidden_1_result, self.weights['log_sigma_bias_1'])

        log_sigma_result = tf.matmul(self.input, self.weights['log_sigma_weights_1'])
        self.z_log_sigma_sq = tf.add(log_sigma_result, self.weights['log_sigma_bias_1'])

        eps = tf.random_normal(tf.stack([tf.shape(self.input)[0], self.hidden_size]), 0, 1, dtype=tf.float32)
        self.z = tf.add(self.z_mean, tf.multiply(tf.sqrt(tf.exp(self.z_log_sigma_sq)), eps))

        y_result = tf.matmul(self.z, self.weights['weights_2'])
        self.y = tf.add(y_result, self.weights['bias_2'])

        reconstruction_loss = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.y, self.input), 2.0))
        latent_loss = -0.5 * tf.reduce_sum(1 + self.z_log_sigma_sq \
                                           - tf.square(self.z_mean) \
                                           - tf.exp(self.z_log_sigma_sq), 1)

        self.loss_fn = tf.reduce_mean(reconstruction_loss + latent_loss)
        self.optimizer_fn = tf.train.AdamOptimizer(learning_rate=0.001)
        self.optimizer = self.optimizer_fn.minimize(self.loss_fn)


Nach der Definition der Modelle folgt der Code um diese zu trainieren. Das Training durchzuführen dauert entsprechend lange, auf einer mittelmässigen GPU benötigt es ca. einen Tag Rechenzeit. Als erstes werden alle benötigten Bibliotheken importiert und einige Hilfs-Funktionen definiert bevor der eigentliche Code für das Training folgt:

In [ ]:
import os
import re
import sys
import tensorflow as tf
import h5py
import time

from os import path
from numpy import random

def camel_to_sneak(name):
    '''Convert a string from camel-case to sneak-case.'''
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

def error(msg):
    log(msg, level='error')

def log(msg, level='info'):
    ts = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
    print('[%s][%s] %s' % (level.upper(), ts, msg))
    if level == 'error': sys.exit(2)

In [ ]:
# Hyperparameters
TRAINING_EPOCHS = 1000
BATCH_SIZE = 128
DISPLAY_EPOCH = 1
DISPLAY_BATCH = 1000
HIDDEN_SIZE = 256
RESULTS_DIR = path.abspath(path.join(path.dirname(__file__), 'results'))

CurrentAutoEncoder = AutoEncoder

def camel_to_sneak(name):
    '''Convert a string from camel-case to sneak-case.'''
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

def error(msg):
    log(msg, level='error')

def log(msg, level='info'):
    ts = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
    print('[%s][%s] %s' % (level.upper(), ts, msg))
    if level == 'error': sys.exit(2)

encoder_type = camel_to_sneak(CurrentAutoEncoder.__name__)
time_stamp = time.strftime('%Y-%m-%d_%H-%M-%S', time.localtime())
result_name = '%s-%s-results/' % (time_stamp, encoder_type)
result_path = path.join(RESULTS_DIR, result_name)

def get_next_batch(train_data, num_batch, random=False, available_idxs=None):
    '''Helper function for retrieving the next batch from train_data.'''
    idxs = None

    if random:
        idxs = [available_idxs.pop(random.randint(0, len(available_idxs))) \
                for _ in range(BATCH_SIZE)]
        idxs = list(sorted(idxs))
    else:
        start_idx = num_batch*BATCH_SIZE
        idxs = range(start_idx, start_idx+BATCH_SIZE)

    batch_data = train_data[idxs]

    if random:
        return (batch_data, available_idxs)
    else:
        return batch_data

train_data_path = argv[0]
loss_track = []

with h5py.File(train_data_path) as train_f:
    train_data = train_f['x']
    input_size = train_data.shape[1]
    num_samples = train_data.shape[0]

    log('Starting training with a %s' % CurrentAutoEncoder.__name__)

    autoencoder = CurrentAutoEncoder(input_size, HIDDEN_SIZE, session=session)
    saver = tf.train.Saver(tf.global_variables(), max_to_keep=3)

    session.run(tf.global_variables_initializer())

    for epoch in range(TRAINING_EPOCHS):
        log('Starting epoch #%d' % (epoch+1))
        num_batches = int(num_samples / BATCH_SIZE)
        avg_loss = 0

        for num_batch in range(num_batches):
            batch_x = get_next_batch(train_data, num_batch)
            loss = autoencoder.batch_fit(batch_x)
            avg_loss += (loss / num_samples) * BATCH_SIZE

            if (num_batch+1) % DISPLAY_BATCH == 0 or (num_batches-num_batch) < 5:
                log('Batch #%d of #%d, loss = %.5f' % (num_batch+1, num_batches, loss))

        if (epoch+1) % DISPLAY_EPOCH == 0 or (epoch+1) == TRAINING_EPOCHS:
            log('Epoch #%d of #%d, loss = %.5f' % (epoch+1, TRAINING_EPOCHS, avg_loss))
            saver.save(session, result_path)

Nachdem der _AutoEncoder_ trainiert wurde kann dieser verwendet werden um damit die Embeddings für die einzelnen Verbrechen zu generieren: